# Notes
These are just notes I used for constructing the TFP joints

In [33]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
tfd = tfp.distributions
tfb = tfp.bijectors
from tensorflow_probability.python.internal import tensorshape_util

In [2]:
import maxentep

In [3]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]])
var = start // 2

In [66]:
class Scatter(tfb.Bijector):
    def __init__(self, indices, output_shape, validate_args=False, name='scatter'):
        super(Scatter, self).__init__(
          validate_args=validate_args,
          forward_min_event_ndims=0,
          name=name)
        self.indices = indices
        self.output_shape = output_shape
    def _forward(self, x):
        if tf.rank(x) > 1:
            return tf.map_fn(lambda x: tf.scatter_nd(self.indices, x, self.output_shape), x)
        else:
            return tf.scatter_nd(self.indices, x, self.output_shape)
    def _inverse(self, y):
        if tf.rank(y) > 1:
            return tf.map_fn(tf.gather_nd(y, self.indices), y)
        else:
            return tf.gather_nd(y, self.indices)
    def _forward_log_det_jacobian(self, x):
        return tf.zeros([], dtype=x.dtype)
    def _inverse_log_det_jacobian(self, y):
        return tf.zeros([], dtype=y.dtype)
    def _forward_event_shape(self, input_shape_tensor):
        batch_shape = input_shape_tensor[:-1]
        return tensorshape_util.concatenate(batch_shape, self.output_shape)
    def _forward_event_shape_tensor(self,input_shape_tensor):
        batch_shape, d = input_shape_tensor[:-1], input_shape_tensor[-1]
        if d != tf.shape(self.indices)[0]:
            raise ValueError('Input Shape must match indices')
        return tf.concat([batch_shape, self.output_shape], axis=0)
    def _inverse_event_shape_tensor(self,input_shape_tensor):
        r = tf.rank(self.output_shape)
        batch_shape, d = input_shape_tensor[:-r], input_shape_tensor[-r]
        if d != self.output_shape:
            raise ValueError('Inverse event shape must be sample as given output shape')
        return tensorshape_util.concatenate(batch_shape, self.indices.shape[:-1] + input_shape_tensor[self.indices.shape[-1]:])
    

In [67]:
b = Scatter([[0,0], [0,1]], [2,2])
u = tf.ones((5,2))
b.forward_event_shape_tensor((5,2))
b.forward(u)

<tf.Tensor: shape=(5, 2, 2), dtype=float32, numpy=
array([[[1., 1.],
        [0., 0.]],

       [[1., 1.],
        [0., 0.]],

       [[1., 1.],
        [0., 0.]],

       [[1., 1.],
        [0., 0.]],

       [[1., 1.],
        [0., 0.]]], dtype=float32)>

In [70]:
b = Scatter([[0,0], [0,1]], [2,2])
j = tfd.TransformedDistribution(
    tfd.Sample(tfd.Normal(0.0, 1.0), 2),
    bijector=b
)
x = j.sample(10)
x

<tf.Tensor: shape=(10, 2, 2), dtype=float32, numpy=
array([[[ 0.27644458, -0.48991972],
        [ 0.        ,  0.        ]],

       [[ 1.6658283 , -1.7342051 ],
        [ 0.        ,  0.        ]],

       [[ 0.46970877,  0.5198215 ],
        [ 0.        ,  0.        ]],

       [[ 0.9658401 ,  2.1367984 ],
        [ 0.        ,  0.        ]],

       [[-0.08178168,  0.5694594 ],
        [ 0.        ,  0.        ]],

       [[-0.5543804 ,  0.23593669],
        [ 0.        ,  0.        ]],

       [[ 0.5502671 ,  1.1709489 ],
        [ 0.        ,  0.        ]],

       [[-0.54536486, -0.15921196],
        [ 0.        ,  0.        ]],

       [[-1.2256647 , -0.1793744 ],
        [ 0.        ,  0.        ]],

       [[-0.13086261, -0.34953183],
        [ 0.        ,  0.        ]]], dtype=float32)>

In [71]:
j.log_prob(x)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-1.9960985, -4.729103 , -2.0832973, -4.5872545, -2.0033631,
       -2.019379 , -2.6748347, -1.9992627, -2.6050916, -1.9075258],
      dtype=float32)>

In [56]:
x = tfd.Sample(tfd.Normal(0.0, 1.0), 2)
x.sample([1])

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.6557338, -1.2860153]], dtype=float32)>

In [57]:
b.forward(x.sample())

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 0.08750823, -0.7579838 ],
       [ 0.        ,  0.        ]], dtype=float32)>

In [26]:
j.sample()

ValueError: Tensor's shape (2, 2) is not compatible with supplied shape (None,)

In [4]:
i =  tf.keras.Input((1,))
l = maxentep.recip_norm_tri_mat_layer(i, start, var, 'test')

(2, 4, 4)


RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
g = tf.constant(start.astype(np.float32))
L = g.shape[0]
trii = np.triu_indices(L, k=1)
# convert to TF format
trii = tf.constant(np.array(trii).T.reshape(-1,2).astype(np.int32))
print(g)

In [7]:
v = tf.gather_nd(g, trii)

In [9]:
tf.map_fn(lambda v: tfd.Deterministic(0.0), v)

ValueError: TypeError: object of type 'Deterministic' has no len()


In [8]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]])
start

array([[ 0., 10.,  0.,  0.],
       [ 0.,  0.,  3.,  0.],
       [ 0.,  0.,  0.,  2.],
       [ 0.,  0.,  0.,  1.]])

In [532]:
# need to compute how to remove diagnal and then put it back in
test = np.zeros((5,5), dtype=np.int32)
other_test = np.zeros((5,5), dtype=np.int32)
test[np.triu_indices_from(test, k=1)] = np.arange(0, 5 * (5 - 1) // 2)
tri_fill = tfp.math.fill_triangular_inverse(test, upper=True).numpy()
print(tri_fill)
test[np.triu_indices_from(test, k=0)] = np.arange(0, 5 * (5 + 1) // 2)
diag = np.diag(test)
diag_fill = tfp.math.fill_triangular_inverse(test, upper=True).numpy()
indices = np.nonzero(diag[:,None] == diag_fill)[1]
tri_fill[indices] = np.arange(5 * (5 - 1) // 2, 5 * (5 + 1) // 2)
print(tri_fill)

[0 0 1 2 3 0 0 4 5 6 9 0 0 7 8]
[10  0  1  2  3 14 11  4  5  6  9 13 12  7  8]


In [533]:
mask = 

SyntaxError: invalid syntax (<ipython-input-533-51cf80055bfa>, line 1)

In [534]:
x = np.concatenate((test[np.triu_indices_from(test, k=1)], np.diag(test)))
print(tfb.Permute(tri_fill).forward(x))
bi = tfb.Chain([tfb.FillTriangular(upper=True), tfb.Permute(tri_fill)])
print(x, bi.forward(x))

tf.Tensor([ 0  1  2  3  4 14  5  6  7  8 13 12  9 10 11], shape=(15,), dtype=int32)
[ 1  2  3  4  6  7  8 10 11 13  0  5  9 12 14] tf.Tensor(
[[ 0  1  2  3  4]
 [ 0  5  6  7  8]
 [ 0  0  9 10 11]
 [ 0  0  0 12 13]
 [ 0  0  0  0 14]], shape=(5, 5), dtype=int32)


In [546]:
def _compute_trans_diagonal(tri_values, indices, shape):
    # deal with batching
    if tf.rank(tri_values) == 2:
        bd = tf.shape(tri_values)[0]        
        il = indices.shape[0]
        indices = tf.tile(tf.reshape(indices,(1, -1, 2)), [bd, 1, 1])
        # add batch index
        batch_i = tf.reshape(tf.tile(tf.range(bd)[:,tf.newaxis], [1,il]),(bd,-1,1))
        indices = tf.concat((batch_i, indices), axis=-1)
        shape = [bd, *shape]
    m = tf.scatter_nd(indices, tri_values, shape)
    diag = 1 - tf.reduce_sum(m, axis=-1)
    return diag

def _fill_tri_permutation(N):
    '''Gives you a permutation so you can re-insert diagonals that were removed from upper right triangular matrix
       using the fill_triangular bijector
    '''
    # TO TEST:
    # x = np.concatenate((test[np.triu_indices_from(test, k=1)], np.diag(test)))
    # bi = tfb.Chain([tfb.FillTriangular(upper=True), tfb.Permute(tri_fill)])
    # print(bi.forward(x), x)
    test = np.zeros((N,N), dtype=np.int32)
    test[np.triu_indices_from(test, k=1)] = np.arange(0, N * (N - 1) // 2)
    tri_fill = tfp.math.fill_triangular_inverse(test, upper=True).numpy()
    test[np.triu_indices_from(test, k=0)] = np.arange(0, N * (N + 1) // 2)
    diag = np.diag(test)
    diag_fill = tfp.math.fill_triangular_inverse(test, upper=True).numpy()
    indices = np.nonzero(diag[:,None] == diag_fill)[1]
    tri_fill[indices] = np.arange(N * (N - 1) // 2, N * (N + 1) // 2)
    return tri_fill

def build(mat):
    L = mat.shape[0]
    trii = np.triu_indices_from(mat, k=1)
    values = mat[trii].astype(np.float32)
    # convert triangle indices into tf format
    trii = tf.constant(np.array(trii).T.reshape(-1,2).astype(np.int32))
    # need to compute permutation to interleave into a triangular matrix
    perm = _fill_tri_permutation(L)
    # sample values 
    dists = []
    for v in values:
        if v == 0:
            dists.append(tfd.Deterministic(0.))
        else:
            dists.append(
                    tfd.TransformedDistribution(
                        tfd.TruncatedNormal(loc=v, scale=1, low=1., high=1e10),
                        bijector=tfb.Reciprocal()
            ))    
    j = tfd.JointDistributionSequential([
        tfd.Blockwise(dists),
        lambda v: tfd.Independent(
            tfd.Deterministic(loc=_compute_trans_diagonal(v, trii, mat.shape))
            ,1)
    ])
    return tfd.TransformedDistribution(
        tfd.Blockwise(j),
        bijector = tfb.Chain([tfb.FillTriangular(upper=True), tfb.Permute(perm)])
    )


In [547]:
j = build(start)

In [548]:
s = j.sample()
np.round(s.numpy(), 3)

array([[0.897, 0.103, 0.   , 0.   ],
       [0.   , 0.256, 0.744, 0.   ],
       [0.   , 0.   , 0.648, 0.352],
       [0.   , 0.   , 0.   , 1.   ]], dtype=float32)

In [549]:
tf.reduce_sum(s, axis=1)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>

In [550]:
x = j.sample()
j.log_prob(x)

<tf.Tensor: shape=(), dtype=float32, numpy=3.2773397>

In [452]:
j = tfd.Blockwise(tfd.JointDistributionSequential([
    tfd.Independent(tfd.TruncatedNormal(loc=start[:,0], scale=0.1, low=0.0, high=1.),1),
    lambda a: tfd.Independent(tfd.TruncatedNormal(loc=start[:,1], scale=0.1, low=0.0, high=1 - a), 1),
    lambda b,a: tfd.Independent(tfd.TruncatedNormal(loc=start[:,1], scale=0.1, low=0.0, high=1 - a - b), 1),
    lambda c,b,a: tfd.Independent(tfd.Deterministic(loc=1 - a - b - c), 1)]))
j

<tfp.distributions.Blockwise 'Blockwise' batch_shape=[] event_shape=[16] dtype=float64>

In [453]:
j.log_prob(j.sample())
j.sample()

<tf.Tensor: shape=(16,), dtype=float64, numpy=
array([9.61198276e-01, 4.13160981e-02, 1.03990688e-01, 1.28081985e-01,
       2.32366724e-02, 7.71096405e-01, 9.55309238e-02, 5.48507655e-02,
       1.48695270e-02, 1.58146128e-01, 1.03021033e-01, 2.27360476e-02,
       6.95524560e-04, 2.94413688e-02, 6.97457356e-01, 7.94331202e-01])>

In [104]:
np.sum(bj.sample().numpy().reshape(4,4), 0)

array([1., 1., 1., 1.], dtype=float32)

In [ ]:
# sample K transitions at i,j
# scatter fill 